In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

C:\Users\z\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_hdf('../act_2_activity.h5')
df.head().append(df.tail())

,item_id,course_id,video_id,watching_count,video_duration,local_watching_time,video_progress_time,video_start_time,video_end_time,local_start_time,local_end_time
0,U_0,C_course-v1:TsinghuaX+20430064X+sp,V_05f095f52ac6446ba05b883171c66713,1,563.0,563,563.000000,0.000000,563.000000,2017-07-24 13:26:56,2017-07-24 13:36:19
1,U_0,C_course-v1:TsinghuaX+34000888X+sp,V_0d166f12024942fca500891c0346b86a,1,166.0,166,166.000000,0.000000,166.000000,2017-07-06 14:47:00,2017-07-06 14:49:46
2,U_0,C_course-v1:TsinghuaX+00691153X+sp,V_0d2ed7b0f77647388d3ad5420d1d6549,2,415.0,415,414.809998,0.000000,415.000000,2017-07-13 09:01:58,2017-07-13 13:18:15
3,U_0,C_course-v1:TsinghuaX+20430064X+sp,V_10355a6a239c4a1dbc28e99bfb7633c1,1,173.0,174,173.000000,0.000000,173.000000,2017-07-24 13:11:47,2017-07-24 13:14:41
4,U_0,C_course-v1:TsinghuaX+34000888X+sp,V_11f15f23fca948808b5f44e25b75f6a2,1,63.0,19,18.820000,0.000000,18.820000,2017-07-06 14:56:18,2017-07-06 14:56:37
101,U_7295,C_course-v1:XJTU+C00204+sp,V_xjtu-C00204-10-3-7,3,337.0,211,531.327024,1.263000,304.743011,2018-01-04 22:14:48,2018-01-04 22:18:22
102,U_7295,C_course-v1:XJTU+C00204+sp,V_xjtu-C00204-10-3-8,1,252.0,126,186.936996,65.063004,252.000000,2018-01-04 22:20:41,2018-01-04 22:22:47
103,U_7295,C_course-v1:XJTU+C00204+sp,V_xjtu-C00204-10-3-9,2,543.0,368,288.401993,253.100998,541.502991,2018-01-04 22:22:47,2018-01-04 22:28:55
104,U_7295,C_course-v1:XJTU+C00204+sp,V_xjtu-C00204-10-4-10,1,565.0,269,400.863007,0.000000,400.863007,2018-01-04 22:29:00,2018-01-04 22:33:29
105,U_7295,C_course-v1:XJTU+C00204+sp,V_xjtu-C00204-2-1-1,1,420.0,41,50.983002,0.000000,50.983002,2017-11-07 19:03:57,2017-11-07 19:04:38


In [3]:
df.shape

(581676, 11)

In [4]:
df.drop_duplicates(['item_id', 'course_id', 'video_id']).shape

(579277, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581676 entries, 0 to 105
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   item_id              581676 non-null  object 
 1   course_id            581676 non-null  object 
 2   video_id             581676 non-null  object 
 3   watching_count       581676 non-null  int64  
 4   video_duration       581676 non-null  float64
 5   local_watching_time  581676 non-null  int64  
 6   video_progress_time  581676 non-null  float64
 7   video_start_time     581676 non-null  float64
 8   video_end_time       581676 non-null  float64
 9   local_start_time     581676 non-null  object 
 10  local_end_time       581676 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 53.3+ MB


In [6]:
df['local_start_time'] = pd.to_datetime(df['local_start_time'])
df['local_end_time'] = pd.to_datetime(df['local_end_time'])
gc.collect()

18

In [ ]:
uid1 = ['item_id', 'course_id', 'video_id']
uid2 = ['item_id', 'course_id']

# video维度

In [ ]:
# video_start_time, video_end_time, local_start_time, local_end_time
df_video = df.groupby(uid1, as_index=False).agg({
    'watching_count': np.sum,
    'video_duration': np.max,
    'local_watching_time': np.sum,
    'video_progress_time': np.sum,
    'video_start_time': np.min,
    'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max
})

gc.collect()

In [ ]:
# 每个video观看次数
df_video['video_cnt'] = df.groupby(uid1, as_index=False)['watching_count'].count()

In [ ]:
# 最晚的进度条时间点-最早的进度条时间点
df_video['video_watching_duration'] = df_video['video_end_time'] - df_video['video_start_time']

# 观看结束时间-观看开始时间
df_video['local_watching_duration'] = df_video.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

In [ ]:
# 进度条是否读到最后，小于0
df_video['video_is_end'] = df_video['video_end_time'] - df_video['video_duration']

# 一开始是否拖到进度条
df_video['video_is_start'] = df_video['video_start_time'].map(lambda x: 1 if x > 0 else 0)

# 进度条是否从开始读到最后，小于0
df_video['video_is_start_end'] = df_video['video_watching_duration'] - df_video['video_duration']

# 进度条拖动的秒速 - 暂停时长
df_video['forward_seconds'] = df_video['video_watching_duration'] - df_video['video_progress_time']

# 暂停时长，
# df[''] = df[''] - df['']

# 实际观看时长是否有差误
df_video['local_watching_diff'] = df_video['local_watching_duration'] - df_video['local_watching_time']

# 倍速
df_video['speed'] = df_video['local_watching_time'] / (df_video['video_progress_time'] + 0.001)

In [ ]:
df_video.columns

# course维度

In [8]:
# video_start_time, video_end_time, local_start_time, local_end_time
df_course = df_video.groupby(uid1, as_index=False).agg({
    # 'watching_count': np.sum,
    # 'video_duration': np.max,
    # 'local_watching_time': np.sum,
    # 'video_progress_time': np.sum,
    'video_start_time': np.min,
    'video_end_time': np.max,
    # 'local_start_time': np.min,
    # 'local_end_time': np.max
})

df_course.shape

(579277, 11)

In [ ]:
# 课程：观看结束时间-观看开始时间
df_course['local_watching_duration'] = df_course.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

In [ ]:
# 进度条是否读到最后，小于0
data['video_is_end'] = data['video_end_time'] - data['video_duration']

# 一开始是否拖到进度条
data['video_is_start'] = data['video_start_time'].map(lambda x: 1 if x > 0 else 0)

# 进度条是否从开始读到最后，小于0
data['video_is_start_end'] = data['video_watching_duration'] - data['video_duration']

# 进度条拖动的秒速 - 暂停时长
data['forward_seconds'] = data['video_watching_duration'] - data['video_progress_time']

# 暂停时长，
# df[''] = df[''] - df['']

# 实际观看时长是否有差误
data['local_watching_diff'] = data['local_watching_duration'] - data['local_watching_time']

# 倍速
data['speed'] = data['local_watching_time'] / (data['video_progress_time'] + 0.001)

In [ ]:
data.columns

In [ ]:
data.drop(['video_start_time', 'video_end_time', 'video_id'], axis=1, inplace=True)
gc.collect()

In [ ]:
# video_start_time, video_end_time, local_start_time, local_end_time
tmp = data.groupby(uid, as_index=False).agg({
    # 'watching_count': np.sum,
    # 'video_duration': np.sum,
    # 'local_watching_time': np.sum,
    # 'video_progress_time': np.sum,
    # 'video_start_time': np.min,
    # 'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max,
    'video_is_end': np.sum,
    'video_is_start': np.mean,
    'video_is_start_end': np.sum,
    'forward_seconds': np.sum,
    'local_watching_diff': np.mean,
    'speed': np.mean
})
data = data.merge(tmp, on=uid, how='left')
del tmp
gc.collect()

In [ ]:
# watching_count
tmp = data.groupby(uid, as_index=False)['watching_count'].agg({
    'watching_count_sum': 'sum',
    'watching_count_count': 'count',
    'watching_count_mean': 'mean',
    'watching_count_max': 'max',
    'watching_count_min': 'min',
    'watching_count_std': 'std',
    'watching_count_skew': 'skew',
    # 'watching_count_mad': 'mad'
})
data = data.merge(tmp, on=uid, how='left')
del tmp
gc.collect()

In [ ]:
# video_duration
tmp = data.groupby(uid, as_index=False)['video_duration'].agg({
    'video_duration_sum': 'sum',
    'video_duration_count': 'count',
    'video_duration_mean': 'mean',
    'video_duration_max': 'max',
    'video_duration_min': 'min',
    'video_duration_std': 'std',
    'video_duration_skew': 'skew',
    # 'video_duration_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid, how='left')
del tmp
gc.collect()

In [ ]:
# local_watching_time
tmp = data.groupby(uid, as_index=False)['local_watching_time'].agg({
    'local_watching_time_sum': 'sum',
    'local_watching_time_count': 'count',
    'local_watching_time_mean': 'mean',
    'local_watching_time_max': 'max',
    'local_watching_time_min': 'min',
    'local_watching_time_std': 'std',
    'local_watching_time_skew': 'skew',
    # 'local_watching_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid, how='left')
del tmp
gc.collect()

In [ ]:
# video_progress_time
tmp = data.groupby(uid, as_index=False)['video_progress_time'].agg({
    'video_progress_time_sum': 'sum',
    'video_progress_time_count': 'count',
    'video_progress_time_mean': 'mean',
    'video_progress_time_max': 'max',
    'video_progress_time_min': 'min',
    'video_progress_time_std': 'std',
    'video_progress_time_skew': 'skew',
    # 'video_progress_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid, how='left')
del tmp
gc.collect()

In [ ]:
# 课程：观看结束时间-观看开始时间
data2['local_watching_duration'] = data2.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

data2['local_watching_diff'] = data2['local_watching_duration'] - data2['local_watching_time_sum']

In [ ]:
data2.drop(['local_start_time', 'local_end_time'], axis=1, inplace=True)
del data
gc.collect()